In [ ]:
%cd /content/
!pip install --upgrade pip
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118

# Installing TinyCuda
%cd /content/
!gdown "https://drive.google.com/u/1/uc?id=1-7x7qQfB7bIw2zV4Lr6-yhvMpjXC84Q5&confirm=t"
!pip install tinycudann-1.7-cp310-cp310-linux_x86_64.whl

# Install nerfstudio
%cd /content/
!pip install git+https://github.com/nerfstudio-project/nerfstudio.git

!git clone https://github.com/alaa-khamis/looping-gifs-nerf.git

In [ ]:
#Load Data
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

scene = 'myvid'

Mounted at /content/drive


In [ ]:
import os
from google.colab import files
from IPython.core.display import display, HTML

scene = '\uD83D\uDDBC poster' #@param ['📤 upload your images' , '🎥 upload your own video']
scene = ' '.join(scene.split(' ')[1:])

display(HTML('Select your custom data'))
display(HTML('You can select multiple images by pressing ctrl, cmd or shift and click.'))
display(HTML('Note: This may take time, especially on higher resolution inputs, so we recommend to download dataset after creation.'))
!mkdir -p /content/data/nerfstudio/custom_data

if scene == 'upload your images':
    !mkdir -p /content/data/images
    %cd /content/data/images
    uploaded = files.upload()
    dir = os.getcwd()
else:
    %cd /content/data/video
    uploaded = files.upload()
    dir = os.getcwd()

preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
del uploaded

%cd /content/data/

if scene == 'upload your images':
    !python colmap2nerf.py --colmap_matcher exhaustive --run_colmap --aabb_scale 32 
else:
    video_path = preupload_datasets[0]
    !python colmap2nerf.py --video_in $video_path --video_fps 2 --run_colmap --aabb_scale 16

scene = "custom_data"

In [ ]:

%cd /content

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(3) # the previous command needs time to write to url.txt


with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# from nerfstudio.utils.io import load_from_json
# from pathlib import Path
# json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# version = load_from_json(Path(json_filename))["version"]
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
print("You may need to click Refresh Page after you start training!")
from IPython import display
display.IFrame(src=url, height=800, width="100%")

In [ ]:
import os

%cd /content
if os.path.exists(f"drive/MyDrive/{scene}/transforms.json"):
    !ns-train nerfacto --viewer.websocket-port 7007 nerfstudio-data --data drive/MyDrive/$scene
else:
    from IPython.core.display import display, HTML
    display(HTML('Error: Data processing did not complete'))
    display(HTML('Please re-run `Downloading and Processing Data`, or view the FAQ for more info.'))

In [ ]:
%cd /content/
!ns-render camera-path --load-config drive/MyDrive/nerfacto/2023-07-22_190553/config.yml --camera-path-filename drive/MyDrive/nerfacto/2023-07-22_190553/camera_paths/camera_path.json --output-path renders/output.mp4